# Data Loading
--- 
Extract latest data from Zenodo Datastore

In [ ]:
# from lib.workspace.zenodo_client import Zenodo
import os

from dotenv import load_dotenv
load_dotenv()

# zenodo_datastore = Zenodo(os.getenv("ZENODO_TOKEN"))
# published_depositions = zenodo_datastore.list_depositions()

# deposition_ids = [file['id'] for file in published_depositions]
# file_urls = [f'https://zenodo.org/api/records/{deposition_id}' for deposition_id in deposition_ids]
# print(file_urls) 

['https://zenodo.org/api/records/14903859', 'https://zenodo.org/api/records/14901033', 'https://zenodo.org/api/records/14900942', 'https://zenodo.org/api/records/14900714', 'https://zenodo.org/api/records/14898218', 'https://zenodo.org/api/records/14890740', 'https://zenodo.org/api/records/14881009', 'https://zenodo.org/api/records/14877594', 'https://zenodo.org/api/records/14874096', 'https://zenodo.org/api/records/14872284', 'https://zenodo.org/api/records/14871202', 'https://zenodo.org/api/records/14870313', 'https://zenodo.org/api/records/14869934', 'https://zenodo.org/api/records/14868704']


## Available datasets on Zenodo

In [2]:
from datetime import date
import requests

# Iterate over each deposition_id in the file_urls list
file_links = []
for url in file_urls: 
    # Send a GET request to the URL
    response = requests.get(url).json()['files']
    link = "".join([file['links']['self'] for file in response])
    file_links.append(link)

print(file_links)

['https://zenodo.org/api/records/14903859/files/rent_contracts_2025-02-21.parquet/content', 'https://zenodo.org/api/records/14901033/files/rent_contracts_2025-02-20.parquet/content', 'https://zenodo.org/api/records/14900942/files/rent_contracts_2025-02-20.parquet/content', 'https://zenodo.org/api/records/14900714/files/rent_contracts_2025-02-20.parquet/content', 'https://zenodo.org/api/records/14898218/files/rent_contracts_2025-02-20.parquet/content', 'https://zenodo.org/api/records/14890740/files/rent_contracts_2025-02-19.parquet/content', 'https://zenodo.org/api/records/14881009/files/rent_contracts_2025-02-17.parquet/content', 'https://zenodo.org/api/records/14877594/files/rent_contracts_2025-02-16.parquet/content', 'https://zenodo.org/api/records/14874096/files/rent_contracts_2025-02-15.parquet/content', 'https://zenodo.org/api/records/14872284/files/rent_contracts_2025-02-14.parquet/content', 'https://zenodo.org/api/records/14871202/files/rent_contracts_2025-02-14.parquet/content'

# Data Exporation 
--- 
Schema

In [3]:
import polars as pl
lf = pl.scan_parquet(file_links[0])
lf.collect_schema()


Schema([('literal', List(String))])